# Поиск максимума функции $\phi\left(\tau,F_0,t_1\right)$

$$\phi\left(\tau,F_0,t_1\right)=\tau F_0\exp\left(-\frac{0.05}{\tau F_0}\right)\left[1-\exp\left(\frac{t_1 F_0-0.45}{\tau F_0}\right)-\exp\left(-\frac{0.5}{\tau F_0}\right)+\exp\left(\frac{t_1 F_0-0.95}{\tau F_0}\right)\right]$$

То есть, при фиксированных $t_1$ и $\tau$ алгоритм находит значение $F_0$, при котором достигается максимум $\phi\left(\tau,F_0,t_1\right)$.
Затем, вычисляется значение $\phi\left(\tau,F_0,t_1\right)$ при найденном $F_0$.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def get_phi_max(t_const, init_freq_pwr, f_pul, n_iters=200, learning_rate=0.1):
    
    def get_phi(time_constant, filling_pulse, frequency_pwr):
        frequency = tf.pow(10.0, frequency_pwr)
        
        a = time_constant * frequency
        b = filling_pulse * frequency
        
        exp0 = tf.exp(-0.05 / (a))
        exp1 = tf.exp((b - 0.45) / (a))
        exp2 = tf.exp(-0.5 / (a))
        exp3 = tf.exp((b - 0.95) / (a))

        phi = a * exp0 * (1.0 - exp1 - exp2 + exp3)
        
        return phi
    
        
    time_constant = tf.Variable(t_const, trainable=False)
    filling_pulse = tf.Variable(f_pul, trainable=False)
    frequency_pwr = tf.Variable(init_freq_pwr, trainable=True)

    for _ in range(n_iters):
        with tf.GradientTape() as t:
            current_loss = 0.0 - get_phi(time_constant, filling_pulse, frequency_pwr)

        d_freq_pwr = t.gradient(current_loss, frequency_pwr)
        frequency_pwr.assign_sub(learning_rate * d_freq_pwr)

    return frequency_pwr.numpy(), get_phi(time_constant, filling_pulse, frequency_pwr).numpy()

In [3]:
def report_n_iter(freq_pwrs, phis, start=0, number=30):
    end = start + number
    for i, (pwr, phi) in enumerate(zip(freq_pwrs[start: end], phis[start: end])):
        print(f'#{i+start:3d}: tau = {taus[i]:1.5f}, freq_pwr = {pwr:1.5f}, freq = {10**pwr:3.5f}, phi = {phi}, 1/phi = {1/phi}')

In [4]:
start = np.log10(0.0002)
end = np.log10(0.25)
tau_pwrs = np.linspace(start, end, num=10)
taus = np.power(10, tau_pwrs)

freq_pwrs, phis = list(), list()

for tau in taus:
    freq_pwr, phi = get_phi_max(t_const=float(tau), 
                                init_freq_pwr=float(np.log10(1/tau)), 
                                f_pul=20 * 10 ** (-6))
    
    freq_pwrs.append(freq_pwr)
    phis.append(phi)
    
report_n_iter(freq_pwrs, phis, number=10)

#  0: tau = 0.00020, freq_pwr = 3.29817, freq = 1986.86201, phi = 0.16149884462356567, 1/phi = 6.191994762135168
#  1: tau = 0.00044, freq_pwr = 2.97832, freq = 951.31463, phi = 0.16648845374584198, 1/phi = 6.006422532619472
#  2: tau = 0.00098, freq_pwr = 2.64566, freq = 442.24388, phi = 0.16881603002548218, 1/phi = 5.923608083006416
#  3: tau = 0.00215, freq_pwr = 2.30684, freq = 202.69333, phi = 0.16988438367843628, 1/phi = 5.886356228556232
#  4: tau = 0.00476, freq_pwr = 1.96515, freq = 92.28873, phi = 0.17037105560302734, 1/phi = 5.869541609990652
#  5: tau = 0.01051, freq_pwr = 1.62214, freq = 41.89322, phi = 0.1705920398235321, 1/phi = 5.86193823014511
#  6: tau = 0.02321, freq_pwr = 1.27854, freq = 18.99065, phi = 0.17069225013256073, 1/phi = 5.858496793049441
#  7: tau = 0.05126, freq_pwr = 0.93466, freq = 8.60328, phi = 0.1707376390695572, 1/phi = 5.856939368785624
#  8: tau = 0.11320, freq_pwr = 0.59066, freq = 3.89641, phi = 0.1707581877708435, 1/phi = 5.856234556330582
# 